In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
#pull in csv from WeatherPY 
cities_csv = "cities.csv"
cities_df = pd.read_csv(cities_csv)
cities_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,duku,11.2361,4.9073,32.60,13,80,1.57
1,1,ushuaia,-54.8000,-68.3000,11.00,62,20,9.26
2,2,karratha,-20.7377,116.8463,27.30,65,6,1.39
3,3,rikitea,-23.1203,-134.9692,26.67,75,6,5.44
4,4,atuona,-9.8000,-139.0333,27.05,70,6,7.91
...,...,...,...,...,...,...,...,...
587,637,ouallam,14.3188,2.0866,31.00,11,0,6.10
588,638,rio grande,-32.0350,-52.0986,22.89,73,40,4.12
589,639,pangai,-19.8000,-174.3500,29.00,79,20,1.54
590,640,impfondo,1.6381,18.0667,21.00,92,33,1.15


In [4]:
#configure gmaps
gmaps.configure(api_key = g_key)

locations = cities_df[["Latitude", "Longitude"]].astype(float)
humid = cities_df["Humidity"].astype(float)

In [5]:
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig = gmaps.figure()
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
ideal_city = cities_df.loc[((cities_df["Temperature"] > 21) & (cities_df["Temperature"] < 28) & (cities_df["Cloudiness"] <15) & 
              (cities_df["Humidity"] <50) & (cities_df["Wind Speed"]<5))]
ideal_city["Location"] = ideal_city["Latitude"].astype(str) + "," + ideal_city["Longitude"].astype(str)

<ipython-input-6-ad29fb2e38bf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_city["Location"] = ideal_city["Latitude"].astype(str) + "," + ideal_city["Longitude"].astype(str)


In [7]:
ideal_city["Hotel Name"] =""
ideal_city

<ipython-input-7-2380006336d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_city["Hotel Name"] =""


,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Location,Hotel Name
17,18,altar,30.7000,-111.8167,22.78,12,0,4.47,"30.7,-111.8167",
181,197,asyut,27.1810,31.1837,25.00,27,0,3.09,"27.180999999999997,31.1837",
200,216,coahuayana,18.7333,-103.6833,27.00,39,1,2.71,"18.7333,-103.6833",
246,264,lazaro cardenas,17.9583,-102.2000,27.49,46,0,4.67,"17.9583,-102.2",
296,321,san rafael,-34.6177,-68.3301,21.80,37,0,4.69,"-34.6177,-68.3301",
335,362,jashpurnagar,22.9000,84.1500,21.29,23,0,2.02,"22.9,84.15",
360,387,san juan,-31.5375,-68.5364,23.16,35,0,3.02,"-31.5375,-68.5364",
394,425,vallenar,-28.5708,-70.7581,23.23,33,0,2.04,"-28.5708,-70.7581",
399,430,zitlala,17.6333,-99.0833,23.79,29,6,4.87,"17.6333,-99.0833",
488,526,shwebo,22.5667,95.7000,25.94,29,0,2.65,"22.5667,95.7",


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}

loc = ideal_city["Location"]


for index, row in ideal_city.iterrows():
    params["location"] = loc[index] 
    response = requests.get(base_url, params=params).json()
    #ideal_city.loc[index, 'Hotel Name'] = results[0]['name']
    print(response)


{'html_attributions': [], 'next_page_token': 'ATtYBwIdbm-YJyPdU2l2IhXho0LEQ_9dOa219_0Gj41Y9rgHCDs7sGuHyiMh_UACJkXRsnXm8dZlpOG0VHDbGvs5QqDjr47B00cuDamR0qXTuf5bvGSE9fdyjhrjVP7cBU0ovgcWgZ8KAzT6Y8PHGWSxzCjM_G2iZrkBRHiqMAV1etjz9bILOXUOaAkp65X7OZlfGK0LlhkZ4PVVP8nPPSlqobJe-PEP0PZD4ir86tSWfMIsYshMoDVYovQRUqkjuUDK3rtC4aFzLLEoFtDWKuQT2NSmr9JAHeHn5mwc7tioOnXLamnrHhfoHdkA9i2INjjoaLRhQJnxAKu9zE06EpWdoA0rBX19ORwXLZMkg3x5Nx4zn9IJj5RPHMEnmvZUJ40jHd3JqM5mMywF5YZVJEI-ysXKoOM8wBJXKFo3KDSgqJXQvv22D_ae0aj7', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 30.7048166, 'lng': -111.8230222}, 'viewport': {'northeast': {'lat': 30.70616558029149, 'lng': -111.8216732197085}, 'southwest': {'lat': 30.7034676197085, 'lng': -111.8243711802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'name': 'La Capital Del Sabor', 'place_id': 'ChIJAeu7r6On04YRvQ8Ns4cXdT4', 'plus_code': {'compound_code': 'P53G+WQ Altar Municipality, Sonora, Mexico', 'g

{'html_attributions': [], 'next_page_token': 'ATtYBwJbv5UHziqnPQKJMiaZnqqIa__trkcJYmdLyWV-Lg15TmAOQF6I50ZlMi2btaeM01XUM71U1RId7u72JnokZoyAtsEYXWsCL1KabM-6L-bwdYigl-_AXMrhpluXV9ZAE2yWqzsAtMaFIouRQQZwOr6bcmIYy8sUTyTQRgVamTwEz19cwJ22ZHNkM84U6uITdQ4M9X7Ucxr8E5W1UZpEwkTEzzPnf3FDWQaTdDhfxhQxXBVeipbbyYvtQsJ4vk3Lu7M1EeupWRQswOL-9Ey8YoQ2-M7YJldXzBnFIXIqp5V3xO5QLfixNryl357_5YHuzkg7dEhNIFYKxoLfB8uxDQ8wJ23Hii6udXutT7JiufUQ-pmCQYYdTSHD1uda8pvvcSQ3pKPJu-Pqukkg7mpqy21GHUhpLBYeYcQPMwsIipfGRMFtAxnT4GNp', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 27.18122749999999, 'lng': 31.18381179999999}, 'viewport': {'northeast': {'lat': 27.1826317302915, 'lng': 31.1850378802915}, 'southwest': {'lat': 27.1799337697085, 'lng': 31.1823399197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png', 'name': 'برج التجاريين', 'place_id': 'ChIJ8_d8rrELRRQRb3s3dUBSUJM', 'plus_code': {'compound_code': '55JM+FG Asyut First, Egypt', 'global_code': '7GVH

{'html_attributions': [], 'next_page_token': 'ATtYBwKRogHsG5kvyT2KxOe_QgQGy1RnhSzwLK2c2OK3oHTU0TBIb85MreA5eZcoc9Bl6xJfWM3e6As5o7jLTKZNcGhTGOfRR6jXCdTNZ73OCMs4uJKY891JEuDKpHbWN_bx3mkRmq1fmHJ4ckghB7Ed2ZwKxuWfaVhjLkPJEiU-Z7LykeURPY4fcGjx6XErJBlYXH4Qzfail9BkBM7aHQrX73HxkfZK0gfegCsvGUcMSu9g55z7j7SZ-S6reKaZNYBBPsc1n5yz3MFAtcIzRmSOjKF1zCvYwwzMipqgMFJAvFPaOadjjOgUhr25heHNxw8HB62ujKCyzhH4U_aumYXbbVZ8sEOKaYgUorcDfElwfZ7LiGbbEOgkCo6sAnMmJq1KSKBpogcWB6Aluou08KoVHeM39AGsOAo_RojW04t0bkEbdztE87qvVb5X', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 18.7306974, 'lng': -103.6856348}, 'viewport': {'northeast': {'lat': 18.7320804802915, 'lng': -103.6843391697085}, 'southwest': {'lat': 18.7293825197085, 'lng': -103.6870371302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png', 'name': 'campo de futbol', 'opening_hours': {'open_now': True}, 'place_id': 'ChIJnV4PqwGjOoQROvVTyyTk0Y0', 'plus_code': {'compound_code': 'P8J7+7P Sa

{'html_attributions': [], 'next_page_token': 'ATtYBwLYuL3IZ63-1QybAZBICTALxJnCII_4H11FJyIPJygX6IMTMfbjeIuyKviY5_hicZjXESoWzYNcDh7W2WuUGvSBEpxn3KSHlgYpt4xTknnrZYcbLyj4VOKI1aeN59rkmoEI7Xifc6iKcOfG0Inp5DVYRTGD3iHLs7NGuPkz0y9NGghyiS1BLnFoJI42KFRIfZlJ9Z8FrcDU4sNA10BrwcITIuhV8V8sSwaoWAVHX0h4IG3RMh051HZGUzfmk7rMAvzFLQM8VctaeRt1-hBaQlUPwmOwHrww1-fVuG4HJJx0iB6RMDMoHp9_khWnflbQyoTiQLBkXuB9XpunQ7f82g2tg8TZc8jFeDd-JMfRKcbkOpEWmzNQsbdnprVXBmyehZ7l02c9RWFXPn23NY_XNb8ZHoxPSNRrWycjWgGP9VDU8IP9LbPJBMLf', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 17.958326, 'lng': -102.2001007}, 'viewport': {'northeast': {'lat': 17.9596440302915, 'lng': -102.1987967697085}, 'southwest': {'lat': 17.9569460697085, 'lng': -102.2014947302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png', 'name': 'Bodega de tupper ware porteo', 'place_id': 'ChIJYX9AIWVcMYQRG7N_y06e02w', 'plus_code': {'compound_code': 'XQ5X+8X Lázaro Cárdenas, Michoacán,

{'html_attributions': [], 'next_page_token': 'ATtYBwIzvAU1V--bDFqaogh3535RfZioiUmL96XHDT7avLfFbM2bYVuEOWRQn_Lxrk0vYyjPkdqcqtU9XOQqXfzrgTWRVYrtRJGnX-eP7L-c4cw8YRbM5C-sAcblHvpv3KBWxWmRPlOTlr__ZeMPnts5PKRtuPzRJYdY9OZmAixMwttEmvjt2MmU7muep6it8RPP4j6_tNdGPRQF5Z3l92l-blvtT7JiE-Efy59lvbeDW-Ty9e6fhoJ3VQt_bQcQJ5Y1hGSXJ9_DO4fGP3G2RGpVa9b5QtluCMJo_JZQ04hqGJZQO97L23fsWkPKHrgWtEcTYNZbBhDK0Y6GYF9HFdhcx6s-aM0jh9EiwjPUVRdFcC5sRlIZ6NULr7jmD6qcWl3AMAH6uGwxHYwH0KFh44oeHhXwsSpBvrKBv_yfprI1W5pmd0fdTHD1VJKn', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -34.61747499999999, 'lng': -68.3302499}, 'viewport': {'northeast': {'lat': -34.6160547197085, 'lng': -68.3288653697085}, 'southwest': {'lat': -34.6187526802915, 'lng': -68.3315633302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/school-71.png', 'name': 'Mixed Normal School', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2592, 'html_attributions': ['<a href="https://maps.google.com/

...

{'html_attributions': [], 'next_page_token': 'ATtYBwLTQ_R-ATTKiR2RWx4F-RmXfDKbIwjrbVqB_MqyxEJHWtKszA5q4lpdnTA0EWpBRonwliwSEdtVC7oigGVNG6_ln-mWhI8_aCoPOHEY2bNXi1dZzmMJiic0xuWJhOA3SJmqrpBvcP_HgT3F-EM1Iyovsb-6SaIzNq4r-33wFs6psrUaG6SUX7Ci9G_u_qLtLAQSSAmGfXHDGSSeW2NfJmbKtsrBcVnC4kAkMze8k0TnclC7oqQEO2r_yzYzhBKpMv8PeS8XfvG69AoN9ji-lSqOq3s-qXGKzbVwnoYpNxVUIr3zQkxRfxkUOjgYq_WAvt4CurS2b5gfuRa52Zy-UVrvDbykD9DqRhk6eriLsrRex_eFDMvZ8CmYxCC2KUuJ97Qzyh4yO1k1yk8p1OFlyeKFuRTKvz090rUQ7JFJUSGilYDS6bhTxBsU', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -31.5373869, 'lng': -68.536441}, 'viewport': {'northeast': {'lat': -31.5360405197085, 'lng': -68.53512886970849}, 'southwest': {'lat': -31.5387384802915, 'lng': -68.5378268302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png', 'name': 'Salon Viñas', 'photos': [{'height': 520, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113411303747988322492">FEBE CA

{'html_attributions': [], 'next_page_token': 'ATtYBwKDVHTCsQ0HvzOsK0oVrQTQx90CmyLUZqxQdJJhXOOrH_lZaqepb59WKILmDW3de7jsxxQh2RUwb1sxvyDVLhZVqR4xWu-5T6tHKYbwvGwzYRx9-Rs5Jej5eazAz6I8GMW52a62RlQfCQ7JfcnGZBK-bpwXUYbEeM1VxiNUC7wY9StlYk-s4l0kVfu2LvXM4xouQFJc-5GRJ3fm0O2LDB-rxmAm95xoS8uHYaU-XeMEGxNSaJ53ZzdczwsagiPntmU6NvNHmBfCBgoVa7EcpEoFcAt13Aaq49tsEuJ2N89JIu0Qxpo6M-EDrY2kyvM16uKvaIL-0aJXLwl1fHl21UDNKLP9x1xun57MbBiIRETTMtvNbdcGDJUiVaKgNuximtacRYQvrzh52Gsy2oxhq0gFOlZpj9Gxc4Sz1Mc1n_YHWnPPkNQGhqSf', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -28.5712487, 'lng': -70.7584595}, 'viewport': {'northeast': {'lat': -28.5699287197085, 'lng': -70.75712446970849}, 'southwest': {'lat': -28.5726266802915, 'lng': -70.7598224302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png', 'name': 'Juan Manuel Barrera Marin', 'place_id': 'ChIJ8c8vi4FIl5YRy-t15Uq-nJc', 'plus_code': {'compound_code': 'C6HR+GJ Vallenar, Chile', 'global_co

{'html_attributions': [], 'next_page_token': 'ATtYBwKb5ikN3csru4VjiGX19mhcZRc1JmeD309BrrHaik5K6U26WiCfNdeDPp2I6gD8Ag--dUYJOIyCOdnt9k-kSG2rNRHkf49RzTbdICYqGNzTxKIpI65IApObgG8dJfGIViJiuEK5wU2pfarQvt_yq-QUoTFFKg8YF418f14cmyPHyakumv-O-18_N8Ff7spf64slLIqN1nvv807bX_mLp_08iPAHme_gCLCtqZ_i2cRbIab6BjUZ1juPWJVOa6h8vvyH68ofLWv54rIdqU_ydvg1rvNX5_-RZm8-FV5NCWSGCdw4aNqSvJ1ozb2RwsITVbNbeY5gNuyikNRLgbZH7ZwXC369-xKR29gcCbXFxw5szRICEI-oMJQnsRg58Xx9xpu2g7xB8NlktiV0l1dIBnXwCTEaPEI4Lc-0-47WS9OXYAuUf4wohb4J', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 17.6155312, 'lng': -99.09947319999999}, 'viewport': {'northeast': {'lat': 17.6168881302915, 'lng': -99.09809201970847}, 'southwest': {'lat': 17.6141901697085, 'lng': -99.1007899802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png', 'name': 'Xulchuchio', 'photos': [{'height': 3264, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/118130270190587817727">Yasmín Gar

{'html_attributions': [], 'next_page_token': 'ATtYBwKzNE2Hq22i-hPRVHJiygBkavXP09gOY9LH7KiNtipFC8aIH0UB-QLKJAk8kDz8zhiAPk0GzkRhE5ugxojwFiWfYm4ZLkqGTTHL1tnWaaQPSAESeDrgynBW8Jcp4rJveEx147EX5HgMKrLfmNwhTur7Bn6YgpB8PxOUrGip1Yax-FOT9xm8JXHBVXe6kayw6Fwc5E5xFVOJ8-UCkaQDZS-rTx7J5zXepWMdl97Y8tiSd6G4Xg0arQdjC6vySOYshHVyj1r_sTM2BTqDmgLlTeX7k7ZF4KF_loRQRQc3q-F9Xse8QNTEKi6J_f0uCGFr_L5_TAvg4ndYjtgP1HGYNt_nyLS1fQiyDN-eeZFtjzjssTTqCUN3pEWP4aayDzgPhypQh6vekfm5NW3RDkRFkFjdOQ5kY8Xsq7jzkhOsmusZN9kCmI-frDGe', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 22.5667399, 'lng': 95.6998742}, 'viewport': {'northeast': {'lat': 22.56811083029151, 'lng': 95.70122433029151}, 'southwest': {'lat': 22.5654128697085, 'lng': 95.6985263697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/civic_building-71.png', 'name': 'Myowai', 'place_id': 'ChIJJYDDiqQrNTcRa7nybXpOOM0', 'plus_code': {'compound_code': 'HM8X+MW Shwebo, Myanmar (Burma)', 'global_code': '7MJQHM8X+MW'

{'html_attributions': [], 'next_page_token': 'ATtYBwLgwBiHZeebWas6fHuNOhhO_7pg_IXNe9HIVqKbfrZV0kNSXo5G4uQIVSgZ82SSUUsvB7uGWiCK54buJDkGmvuraxBdmITP3jSkFe1H5Onguop5njgcGOLvCNB5kVNaeUM4dzgIG3hsfvCyUubNEogjIQwsNCIe_ckqYZA4TdaCDAWLLvN8Q33MCu36y4VyQxLbB3pC6xxWdwYcuJp7whm2yoBle1AllI53s-t5EdQpE8Ogdmj87lPwMhIlcfDsDUherhPOqCB_cTXciBj6kRAFdM7xQyI5BCceNDq4cNpRyws542gobHpa2Wk1QSZMHePHldc6mW8YBiNW2tn5C545Wnmr7evKYXmqqvyHYix-PgBFW4OujxNsPkLNuE5Du28SJda7aUwY7V5y_Y9CX6DFKZEvDGo50U_3KgY0kH7str11iQcmStR3', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 26.3261235, 'lng': 43.9751993}, 'viewport': {'northeast': {'lat': 26.3275360302915, 'lng': 43.97660803029149}, 'southwest': {'lat': 26.32483806970849, 'lng': 43.9739100697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png', 'name': 'مكتب نور العيون للعدسات', 'opening_hours': {'open_now': False}, 'place_id': 'ChIJj1QkUlxZfxUR927vcaANcig', 'plus_code': {'compound_code': '8XGG+C3 Buray

In [ ]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in ideal_city.iterrows()]
locations = ideal_city[["Latitude", "Longitude"]]

hotel_info